# Active-learning tutorial: Using committee MACE models to study protonated water clusters

1. Load all the modules
2. read the training pool 
3. select random training set of 25 structure from the pool (can be done with np.rand) --> latter exclude these from the pool
4. Train a committee (just to check we can train 2)
5. predict on the training pool and sort max energy error
6. Then we repeat in a for loop.

## To Do 

- for loop everywhere
- avoid using scripts for MACE
- fix E0s

In [ ]:
from IPython.display import Image, display
display(Image(filename='../initial-datasets/zundel/zundel.png'))

## Import modules

In [ ]:
import os, glob, re
import multiprocessing
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ase.io import read, write # read and write structures
# from ase.visualize import view # visualize structures (optional)

# import functions to run this tutorial
from myfunctions import train_mace     # train MACE model
from myfunctions import eval_mace      # evaluate MACE model
from myfunctions import extxyz2energy  # extract energy from extxyz file
from myfunctions import run_qbc        # run Query by Committee
from myfunctions import clean_output   # clean the output of the training

In [ ]:
n_init_train = 20
n_test = 50  
n_committee = 4
parallel = True
md_folder = "md"
init_train_folder = "init-train"
qbc_folder = "qbc-work" # if you modify this, add the new folder to .gitignore
n_iter_qbc = 20
n_add_iter = 20     

In [ ]:
np.random.seed(0)
plt.style.use('notebook.mplstyle')
os.makedirs(f'{init_train_folder}', exist_ok=True)
os.makedirs(f'{init_train_folder}/config', exist_ok=True)
os.makedirs(f'{init_train_folder}/models', exist_ok=True)
os.makedirs(f'{init_train_folder}/eval', exist_ok=True)
os.makedirs(f'{init_train_folder}/structures', exist_ok=True)
os.makedirs(f'{md_folder}', exist_ok=True)

## Select initial training structures

In [ ]:
# Read the all the structures from file
structures = read('../initial-datasets/zundel/train.extxyz', index=':')
print(f'Total number of structures: {len(structures)}')
# view(structures)  # Opens an interactive GUI window to visualize the structures

In [ ]:
# Create the initial training and test sets
selected_indices = np.random.choice(len(structures), size=(n_init_train + n_test), replace=False)
remaining_candidate_idcs = np.delete(np.arange(len(structures)), selected_indices)

indices_train = selected_indices[:n_init_train]
indices_test = selected_indices[n_init_train:]
assert len(indices_train) == n_init_train
assert len(indices_test) == n_test

print(f'\nSelected indices for training: {indices_train}')
print(f'\nSelected indices for test: {indices_test}')

initial_training_set = [structures[i] for i in indices_train]
test_set = [structures[i] for i in indices_test]
remaining_structures = [structures[i] for i in remaining_candidate_idcs]

print(f"\nSaving the initial training set to 'structures/init.train.extxyz'")
write(f'{init_train_folder}/structures/init.train.extxyz', initial_training_set, format='extxyz')

print(f"\nSaving the test set to 'structures/test.extxyz'")
write(f'{init_train_folder}/structures/test.extxyz', test_set, format='extxyz')

print(f"\nSaving the remaining structures to 'structures/remaining.extxyz'")
write(f'{init_train_folder}/structures/remaining.extxyz', remaining_structures, format='extxyz')

## Initial Training

Hyperparameters for the committee members

In [ ]:
# Define different values for each config
seeds = np.random.randint(0, 2**32 - 1, size=n_committee, dtype=np.uint32)
for i in range(n_committee):
    filename = f"{init_train_folder}/config/config.{i}.yml"
    name = f"mace.com={i}"
    
    config_text = f"""
# You can modify the following parameters
num_channels: 4
max_L: 0            # take it larger but not smaller
max_ell: 1          # take it larger but not smaller
correlation: 1      # take it larger but not smaller
num_interactions: 2 # take it larger but not smaller

# ... but you can also modify these ones
r_max: 4.0
batch_size: 4
max_num_epochs: 10000 # this is basically early stopping
patience: 10

# But please, do not modify these parameters!
model: "MACE"
name: "{name}"

model_dir      : "{init_train_folder}/models"
log_dir        : "{init_train_folder}/log"
checkpoints_dir: "{init_train_folder}/checkpoints"
results_dir    : "{init_train_folder}/results"
train_file     : "{init_train_folder}/structures/init.train.extxyz"

energy_key: "REF_energy"
forces_key: "REF_forces"
E0s: "average" # to be fixed
device: cpu
swa: true
seed: {seeds[i]}
restart_latest: False
"""

    with open(filename, "w") as f:
        f.write(config_text)

    print(f"Wrote {filename}")

In [ ]:
# train a committee of MACE models
def train_single_model(n):
    train_mace(f"{init_train_folder}/config/config.{n}.yml")

if parallel: # parallel version: it should take around 25s 
    print(f"Training {n_committee} models in parallel")
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        pool.map(train_single_model, range(n_committee))
        
else: # serial version: it should take around 1m
    print(f"Training {n_committee} models in serial\n")
    for n in range(n_committee):
        print(f"Training model {n}")
        train_single_model(n)
        
clean_output(init_train_folder,n_committee)

## Evaluation

In [ ]:
# ToDo: put files somewhere else
for n in tqdm(range(n_committee)):
    eval_mace(f'{init_train_folder}/models/mace.n={n:d}.model',
              '../initial-datasets/zundel/train.extxyz',
              f'{init_train_folder}/eval/train_{n:02d}.extxyz')

In [ ]:
# read in predicted energies
energies = np.array([extxyz2energy(f'{init_train_folder}/eval/train_{n:02d}.extxyz') for n in tqdm(range(n_committee))])
avg_energy = energies.mean(axis=0)
disagreement = energies.std(axis=0)

In [ ]:
plt.figure(figsize=(6, 3))
for n, e in enumerate(energies):
    plt.plot(e, label=rf'$E_{n:d}$', alpha=0.5,linewidth=0.5)
plt.plot(avg_energy, label=r'$\overline{E}$', color='k',linewidth=0.5)
plt.legend()
plt.xlabel('Data point index')
plt.ylabel('Energy [eV]')
plt.show()

In [ ]:
plt.figure(figsize=(6, 3))
plt.plot(disagreement,color="blue",linewidth=0.5)
plt.xlabel('Data point index')
plt.ylabel(r'$\sigma(E)$ [eV]')
plt.show()

# Select relevant training data via Query by Committee (QbC)

Some text...

In [ ]:
# Define different values for each config
# TODO: make this simpler - the only thing we need to change is the name of the training extxyz file.
# TODO: implement retraining using the refinement workflow using `foundation_model`
os.makedirs(qbc_folder, exist_ok=True)
os.makedirs(f'{qbc_folder}/config', exist_ok=True)
seeds = np.random.randint(0, 2**32 - 1, size=n_committee, dtype=np.uint32)
for i in range(n_committee):
    filename = f"{qbc_folder}/config/config.{i}.yml"
    name = f"mace.com={i}"
    
    config_text = f"""
# You can modify the following parameters
num_channels: 4
max_L: 0            # take it larger but not smaller
max_ell: 1          # take it larger but not smaller
correlation: 1      # take it larger but not smaller
num_interactions: 2 # take it larger but not smaller

# ... but you can also modify these ones
r_max: 4.0
batch_size: 4
max_num_epochs: 10000 # this is basically early stopping
patience: 10

# But please, do not modify these parameters!
model: "MACE"
name: "{name}"

model_dir      : "{qbc_folder}/models"
log_dir        : "{qbc_folder}/log"
checkpoints_dir: "{qbc_folder}/checkpoints"
results_dir    : "{qbc_folder}/results"

train_file: "{qbc_folder}/train-iter.extxyz"
energy_key: "REF_energy"
forces_key: "REF_forces"

E0s: "average" # to be fixed
device: cpu
swa: true
seed: {seeds[i]}
restart_latest: True

"""

    with open(filename, "w") as f:
        f.write(config_text)

    print(f"Wrote {filename}")

In [ ]:
# Attention: this function will not
fns_committee = [f'{init_train_folder}/models/mace.n={n:d}.model' for n in range(n_committee)]
run_qbc(
    fns_committee=fns_committee,                                       # list of MACE models
    fn_candidates=f'{init_train_folder}/structures/remaining.extxyz',  # candidate structures
    test_dataset=f'{init_train_folder}/structures/test.extxyz',        # test set
    n_iter=n_iter_qbc,                                                 # number of QBC iterations
    config=f'{qbc_folder}/config',                                     # folder with config files
    ofolder=qbc_folder,                                                # folder to save the QBC results
    n_add_iter=n_add_iter,                                             # number of structures to add in each iteration
    recalculate_selected=False,                                        # whether to recalculate the selected structures
);
# it should take 13m

### Candidated and Selected

In [ ]:
n_training_structures   = np.arange(n_init_train,n_iter_qbc*n_add_iter+n_init_train,n_add_iter)
sigma = np.loadtxt('qbc-work/disagreement.txt').T

fig, ax = plt.subplots(figsize=(6, 3))  # 1 row, 2 columns

# --- First plot: Selected ---
ax.plot(n_training_structures, sigma[0], '-o', label='Selected', color="blue")
ax.fill_between(n_training_structures,sigma[0]-sigma[2],sigma[0]+sigma[2],color="blue",alpha=0.2,linewidth=0)

# --- Second plot: Candidates ---
ax.plot(n_training_structures, sigma[1], '-o', label='Candidates', color="red")
ax.fill_between(n_training_structures,sigma[1]-sigma[3],sigma[1]+sigma[3],color="red",alpha=0.2,linewidth=0)

ax.legend()
ax.set_xlabel('n. training structures')
ax.set_ylabel(r'$\sigma(E)$ [eV]')
# axs[0].set_title(r"Selected $\sigma(E)$")
plt.tight_layout()
plt.show()

### Test dataset

In [ ]:
energy = None
all_energy = None
for i_iter in range(n_iter_qbc):
    predictions = [None]*n_committee
    for n_model in range(n_committee):
        file = f"{qbc_folder}/eval/test.model={n_model}.iter={i_iter}.extxyz"
        structures = read(file,index=":")
        if energy is None:
            energy = np.zeros((len(structures),n_committee))
            all_energy = np.zeros((n_iter_qbc,len(structures),n_committee))
        energy[:,n_model] = [ atoms.info["MACE_energy"] for atoms in structures ]
    all_energy[i_iter,:,:] = energy
np.save(f"{qbc_folder}/test-energy.npy",all_energy)

In [ ]:
disagreement = np.std(all_energy,axis=2)
disagreement_mean = np.mean(disagreement,axis=1)
disagreement_std  = np.std(disagreement,axis=1)

In [ ]:
n_training_structures   = np.arange(n_init_train,n_iter_qbc*n_add_iter+n_init_train,n_add_iter)

fig, ax = plt.subplots(figsize=(6, 3))  # 1 row, 2 columns

# --- First plot: Test ---
ax.plot(n_training_structures, disagreement_mean, '-o', label='Test', color="green")
#ax.fill_between(n_training_structures,disagreement_mean-disagreement_std,disagreement_mean+disagreement_std,color="blue",alpha=0.2,linewidth=0)

ax.legend()
ax.set_xlabel('n. training structures')
ax.set_ylabel(r'$\sigma(E)$ [eV]')
# axs[0].set_title(r"Selected $\sigma(E)$")
plt.tight_layout()
plt.show()

In [ ]:
print(predictions[0][0].arrays.keys())
print(predictions[0][0].info.keys())

In [ ]:
df = pd.DataFrame(columns=["iter","E-dis","F-dis","E-rmse","F-rmse"])
for file in glob.glob(f"{qbc_folder}/eval/test.*"):
    # print(file)
    match = re.search(r'model=(\d+)\.iter=(\d+)', file)
    model_num = int(match.group(1))
    iter_num = int(match.group(2))

# Run Molecular Dynamics

## Run FHI-aims

In [ ]:
from myfunctions import run_aims

In [ ]:
to_run  = structures[:4]

In [ ]:
%%capture
run_aims(
    structures=to_run,
    folder='aims',
    command=f"mpirun -n 4 /home/stoccoel/codes/FHIaims-polarization/build/polarization-debug/aims.250131.scalapack.mpi.x",
    control="../aims/control.in"
)